In [ ]:
!pip install transformers datasets

In [ ]:
!pip install accelerate==0.20.1

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install --upgrade transformers[torch]  accelerate -U


In [ ]:
!pip install accelerate==0.20.1

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    # is_deepspeed_zero3_enabled
)

def load_custom_dataset(dataset_path: str, split: str = None):
    if os.path.exists(dataset_path):
        return load_dataset('json', data_files=dataset_path, split=split)
    else:
        return load_dataset(dataset_path, split=split)

def load_model_and_tokenizer(
    model_name_or_path: str,
    config_kwargs: dict = None,
    use_fast_tokenizer: bool = True,
    split_special_tokens: bool = True,
    compute_dtype: torch.dtype = torch.float32
):
    config_kwargs = config_kwargs if config_kwargs is not None else {}
    config = AutoConfig.from_pretrained(model_name_or_path, **config_kwargs)
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        config=config,
        torch_dtype=compute_dtype,
        # low_cpu_mem_usage=(not is_deepspeed_zero3_enabled()),
        **config_kwargs
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path,
        use_fast=use_fast_tokenizer,
        split_special_tokens=split_special_tokens,
        **config_kwargs
    )
    return model, tokenizer
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Models/",
    learning_rate=5e-05,
    num_train_epochs=3.0,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    lr_scheduler_type="cosine",
    max_grad_norm=1.0,
    logging_steps=5,
    save_steps=100,
    warmup_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    fp16=True,
    dataloader_num_workers=0,
)
# Load data
data_path = "nvidia/HelpSteer"
train_dataset = load_custom_dataset(data_path)

# Load model and tokenizer
model_path = "mistralai/Mistral-7B-v0.1"
model, tokenizer = load_model_and_tokenizer(model_path)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start training
trainer.train()

In [ ]:
# Load data
data_path = "/content/drive/MyDrive/Models /Adversarial_chatbot_dataset.txt"
train_dataset = load_custom_dataset(data_path)

# Load model and tokenizer
model_path = "mistralai/Mistral-7B-v0.1"
model, tokenizer = load_model_and_tokenizer(model_path)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start training
trainer.train()
